In [ ]:
!pip install /kaggle/input/autograd/ -f ./ --no-index
!pip install /kaggle/input/autogradgamma/ -f ./ --no-index
!pip install /kaggle/input/lifelines/ -f ./ --no-index
!pip install /kaggle/input/ngboost/ -f ./ --no-index

In [ ]:
import numpy as np
import pandas as pd
import pydicom
import os
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import make_scorer
import lightgbm as lgb
import typing as tp
from scipy.optimize import lsq_linear
from sklearn.linear_model import ElasticNet
from tqdm.keras import TqdmCallback

from ngboost.ngboost import NGBoost
from ngboost.learners import default_tree_learner
from ngboost.distns import Normal, LogNormal
from ngboost.scores import MLE

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M

In [ ]:
def seed_everything(seed=2020):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(42)

In [ ]:
ROOT = "../input/osic-pulmonary-fibrosis-progression"

In [ ]:
tr = pd.read_csv(f"{ROOT}/train.csv")
tr.drop_duplicates(keep=False, inplace=True, subset=['Patient','Weeks'])
chunk = pd.read_csv(f"{ROOT}/test.csv")

print("add infos")
sub = pd.read_csv(f"{ROOT}/sample_submission.csv")
sub['Patient'] = sub['Patient_Week'].apply(lambda x:x.split('_')[0])
sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
sub =  sub[['Patient','Weeks','Confidence','Patient_Week']]
sub = sub.merge(chunk.drop('Weeks', axis=1), on="Patient")

In [ ]:
tr['WHERE'] = 'train'
chunk['WHERE'] = 'val'
sub['WHERE'] = 'test'
data = tr.append([chunk, sub])

In [ ]:
data['min_week'] = data['Weeks']
data.loc[data.WHERE=='test','min_week'] = np.nan
data['min_week'] = data.groupby('Patient')['min_week'].transform('min')

In [ ]:
base = data.loc[data.Weeks == data.min_week]
base = base[['Patient','FVC']].copy()
base.columns = ['Patient','min_FVC']
base['nb'] = 1
base['nb'] = base.groupby('Patient')['nb'].transform('cumsum')
base = base[base.nb==1]
base.drop('nb', axis=1, inplace=True)

In [ ]:
data = data.merge(base, on='Patient', how='left')
data['base_week'] = data['Weeks'] - data['min_week']
del base

In [ ]:
COLS = ['Sex','SmokingStatus'] #,'Age'
FE = []
for col in COLS:
    for mod in data[col].unique():
        FE.append(mod)
        data[mod] = (data[col] == mod).astype(int)
#=================

In [ ]:
percent__ = data[data['base_week'] == 0][['Patient', 'Percent']].values
percent__ = dict(percent__)

percent_list__ = []
for d in data.values:
    percent_list__.append(percent__[d[0]])
    
data['percent_base'] = percent_list__

In [ ]:
def preproc_sex_smoking_status(row):
    if row['SmokingStatus'] == 'Currently smokes':
        if row['Sex'] == 'Male':
            return 1
        if row['Sex'] == 'Female':
            return 2
    if row['SmokingStatus'] == 'Ex-smoker':
        if row['Sex'] == 'Male':
            return 3
        if row['Sex'] == 'Female':
            return 4
    if row['SmokingStatus'] == 'Never smoked':
        if row['Sex'] == 'Male':
            return 5
        if row['Sex'] == 'Female':
            return 6

In [ ]:
data['person_type'] = np.nan

for pid, row in enumerate(data.iloc):
    data['person_type'][pid] = preproc_sex_smoking_status(row)

In [ ]:
data = data.join(pd.get_dummies(data['person_type'], prefix='person_type'))

In [ ]:
def to_normalize(cols):
    normalized_cols = []
    for idx, col in enumerate(cols):
        new_col_name = col + '_normalized'
        data[new_col_name] = (data[col] - data[col].min() ) / (data[col].max() - data[col].min())
        normalized_cols.append(new_col_name)
    return normalized_cols

cols_to_normalize = ['Age', 'base_week', 'min_FVC', 'percent_base', 'Weeks']

normalized_cols = to_normalize(cols_to_normalize)

FE2 = ['person_type_1.0', 'person_type_2.0', 'person_type_3.0',
       'person_type_4.0', 'person_type_5.0', 'person_type_6.0'] + normalized_cols

FE3 = ['person_type'] + cols_to_normalize

In [ ]:
tr = data.loc[data.WHERE=='train']
chunk = data.loc[data.WHERE=='val']
sub = data.loc[data.WHERE=='test']
# del data

In [ ]:
tr.shape, chunk.shape, sub.shape

### BASELINE NN 

In [ ]:
def make_model(nh, delta):
    
    C1, C2 = tf.constant(70, dtype='float32'), tf.constant(1000, dtype="float32")
    #=============================#
    def score(y_true, y_pred):
        tf.dtypes.cast(y_true, tf.float32)
        tf.dtypes.cast(y_pred, tf.float32)
        sigma = y_pred[:, 2] - y_pred[:, 0]
        fvc_pred = y_pred[:, 1]

        #sigma_clip = sigma + C1
        sigma_clip = tf.maximum(sigma, C1)
        delta = tf.abs(y_true[:, 0] - fvc_pred)
        delta = tf.minimum(delta, C2)
        sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
        metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
        return K.mean(metric)
    #============================#
    def qloss(y_true, y_pred):
        # Pinball loss for multiple quantiles
        qs = [0.2, 0.5, 0.8]
        q = tf.constant(np.array([qs]), dtype=tf.float32)
        e = y_true - y_pred
        v = tf.maximum(q*e, (q-1)*e)
        return K.mean(v)
    #=============================#
    def mloss(_lambda):
        def loss(y_true, y_pred):
            return _lambda * qloss(y_true, y_pred) + (1 - _lambda)*score(y_true, y_pred)
        return loss
    
    z = L.Input((nh, 1), name="Patient")
    x = L.Dense(100, activation="relu", name="d1")(z)
    conv = L.Conv1D(filters = 2, kernel_size = 2)(x)
    x = L.Flatten()(conv)
    x = L.Dense(100, activation="relu", name="d2")(x)
    p1 = L.Dense(3, activation="relu", name="p1")(x)
    p2 = L.Dense(3, activation="relu", name="p2")(x)
    preds = L.Lambda(lambda x: x[0] + tf.cumsum(x[1], axis=1), 
                     name="preds")([p1, p2])
    
    model = M.Model(z, preds, name="CNN")
    #model.compile(loss=qloss, optimizer="adam", metrics=[score])
    model.compile(loss=mloss(delta), optimizer=tf.keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False), metrics=[score])
    return model

In [ ]:
y = tr['FVC'].values
# z = tr[FE].values
# ze = sub[FE].values
# nh = z.shape[1]

pe1 = np.zeros((sub[FE2].values.shape[0], 2))
pe2 = np.zeros((sub[FE2].values.shape[0], 2))

pred = np.zeros((tr[FE2].values.shape[0], 3))

z2 = tr[FE2].values

ze2 = sub[FE2].values
nh2 = ze2.shape[1]

In [ ]:
def laplace_log_likelihood(actual_fvc, predicted_fvc, confidence, return_values = False):
    """
    Calculates the modified Laplace Log Likelihood score for this competition.
    """
    sd_clipped = np.maximum(confidence, 70)
    delta = np.minimum(np.abs(actual_fvc - predicted_fvc), 1000)
    metric = - np.sqrt(2) * delta / sd_clipped - np.log(np.sqrt(2) * sd_clipped)

    if return_values:
        return metric
    else:
        return np.mean(metric)
    
scorer_210 = make_scorer(lambda y_true, y_pred: laplace_log_likelihood(y_true, y_pred, 210))

In [ ]:
net = make_model(nh2, 1)
print(net.summary())
print(net.count_params())

In [ ]:
class OSICLossForLGBM:
    """
    Custom Loss for LightGBM.
    
    * Objective: return grad & hess of NLL of gaussian"
    * Evaluation: return competition metric
    """
    
    def __init__(self, epsilon: float=1) -> None:
        """Initialize."""
        self.name = "osic_loss"
        self.n_class = 2  # FVC & Confidence
        self.epsilon = epsilon
    
    def __call__(self, preds: np.ndarray, labels: np.ndarray, weight: tp.Optional[np.ndarray]=None) -> float:
        """Calc loss."""
        sigma_clip = np.maximum(preds[:, 1], 70)
        Delta = np.minimum(np.abs(preds[:, 0] - labels), 1000)
        loss_by_sample = - np.sqrt(2) * Delta / sigma_clip - np.log(np.sqrt(2) * sigma_clip)
        loss = np.average(loss_by_sample, weight)
        
        return loss
    
    def _calc_grad_and_hess(
        self, preds: np.ndarray, labels: np.ndarray, weight: tp.Optional[np.ndarray]=None
    ) -> tp.Tuple[np.ndarray]:
        """Calc Grad and Hess"""
        mu = preds[:, 0]
        sigma = preds[:, 1]
        
        sigma_t = np.log(1 + np.exp(sigma))
        grad_sigma_t = 1 / (1 + np.exp(- sigma))
        hess_sigma_t = grad_sigma_t * (1 - grad_sigma_t)
        
        grad = np.zeros_like(preds)
        hess = np.zeros_like(preds)
        grad[:, 0] = - (labels - mu) / sigma_t ** 2
        hess[:, 0] = 1 / sigma_t ** 2
        
        tmp = ((labels - mu) / sigma_t) ** 2
        grad[:, 1] = 1 / sigma_t * (1 - tmp) * grad_sigma_t
        hess[:, 1] = (
            - 1 / sigma_t ** 2 * (1 - 3 * tmp) * grad_sigma_t ** 2
            + 1 / sigma_t * (1 - tmp) * hess_sigma_t
        )
        if weight is not None:
            grad = grad * weight[:, None]
            hess = hess * weight[:, None]
        return grad, hess
    
    def return_loss(self, preds: np.ndarray, data: lgb.Dataset) -> tp.Tuple[str, float, bool]:
        """Return Loss for lightgbm"""
        labels = data.get_label()
        weight = data.get_weight()
        n_example = len(labels)
        
        # # reshape preds: (n_class * n_example,) => (n_class, n_example) =>  (n_example, n_class)
        preds = preds.reshape(self.n_class, n_example).T
        # # calc loss
        loss = self(preds, labels, weight)
        
        return self.name, loss, True
    
    def return_grad_and_hess(self, preds: np.ndarray, data: lgb.Dataset) -> tp.Tuple[np.ndarray]:
        """Return Grad and Hess for lightgbm"""
        labels = data.get_label()
        weight = data.get_weight()
        n_example = len(labels)
        
        # # reshape preds: (n_class * n_example,) => (n_class, n_example) =>  (n_example, n_class)
        preds = preds.reshape(self.n_class, n_example).T
        # # calc grad and hess.
        grad, hess =  self._calc_grad_and_hess(preds, labels, weight)

        # # reshape grad, hess: (n_example, n_class) => (n_class, n_example) => (n_class * n_example,) 
        grad = grad.T.reshape(n_example * self.n_class)
        hess = hess.T.reshape(n_example * self.n_class)
        
        return grad, hess
    
def find_optimal_solution(preds, targets):
    A = np.array(preds).T
    res = lsq_linear(A, targets, lsq_solver='exact', method='trf', tol=1e-5, verbose=2)
    return A.dot(res.x), res.x

In [ ]:
cnt = 0
delta = 0.4
EPOCHS = 2000
BATCH_SIZE = 512
MODELS = 1
NFOLD = 5
gkf = GroupKFold(n_splits = NFOLD)
NFOLD_MODELS = NFOLD * MODELS
val_scores = []
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
                monitor="val_loss",
                factor=0.05,
                patience=100,
                verbose=0,
                mode="auto",
                min_delta=0.0001,
                cooldown=0,
                min_lr=0.0001)
pred = np.zeros((z2.shape[0], 3))
categorical_features = ['person_type']
fold_n = 0

for tr_idx, val_idx in gkf.split(z2, y, data[data.WHERE=='train'].Patient.values):
    fold_n += 1
    seed_everything(42)

    ngb = NGBoost(Base = default_tree_learner, Dist = Normal, Score=MLE, natural_gradient = True, verbose = False).fit(tr[FE3].iloc[tr_idx].values, y[tr_idx])
    ngb_val_pred = ngb.predict(tr[FE3].iloc[val_idx].values)
    ngb_val_dists = ngb.pred_dist(tr[FE3].iloc[val_idx].values, 1)
    a, b = ngb_val_dists.dist.interval(0.1)
    ngb_val_conf = b - a
    
    ngb_subm_pred = ngb.predict(sub[FE3].values)
    ngb_subm_dists = ngb.pred_dist(sub[FE3].values, 1)
    a, b = ngb_subm_dists.dist.interval(0.1)
    ngb_subm_conf = b - a
    
    net = make_model(nh2, delta)
    net.fit(z2[tr_idx], y[tr_idx], batch_size=BATCH_SIZE, epochs=EPOCHS, 
            validation_data=(z2[val_idx], y[val_idx]), verbose=0, callbacks = [TqdmCallback(verbose = 0), lr_scheduler])
    pred[val_idx] = net.predict(z2[val_idx], batch_size=BATCH_SIZE, verbose=0)

    nn_val_pred = pred[val_idx]
    nn_val_pred_conf = nn_val_pred[:, 2] - nn_val_pred[:, 0]
    nn_val_loss = laplace_log_likelihood(nn_val_pred[:, 1], y[val_idx], nn_val_pred_conf)
        
    train_data = lgb.Dataset(tr[FE3].iloc[tr_idx], label = y[tr_idx], categorical_feature = categorical_features)
    test_data = lgb.Dataset(tr[FE3].iloc[val_idx], label = y[val_idx], categorical_feature = categorical_features)
    
    lgb_model_param = {
    'num_class': 2,
    'metric': 'None',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'seed': 42,
    "subsample": 0.4,
    "subsample_freq": 1,
    'max_depth': 1,
    'verbosity': 0    
    }
    
    lgb_fit_param = {
        "num_boost_round": 500000,
        "verbose_eval": 5000,
        "early_stopping_rounds": 500,
    }
    
    loss = OSICLossForLGBM()
    
    model = lgb.train(lgb_model_param,
                       train_data,
                       valid_sets = test_data,
                       **lgb_fit_param, 
                      fobj=loss.return_grad_and_hess,
                      feval=loss.return_loss)
    lgb_val_predict = model.predict(tr[FE3].iloc[val_idx].values)
    lgb_val_loss = laplace_log_likelihood(lgb_val_predict[:, 0], y[val_idx], nn_val_pred_conf)
    lgb_val_loss_conf = laplace_log_likelihood(lgb_val_predict[:, 0], y[val_idx], lgb_val_predict[:, 1])
    predicted = model.predict(sub[FE3].values)
    
    el_fitted = ElasticNet(alpha=0.3, l1_ratio = 0.8).fit(z2[tr_idx], y[tr_idx])
    el = el_fitted.predict(z2[val_idx])
    el_subm = el_fitted.predict(ze2)
    
    print(f"Loss Keras #{fold_n}: {nn_val_loss}")
    print(f"Loss LGBM #{fold_n}: {lgb_val_loss}, {lgb_val_loss_conf}")
    
    optimal = find_optimal_solution([nn_val_pred[:, 1], lgb_val_predict[:, 0], el, ngb_val_pred], y[val_idx])
    optimal_loss = laplace_log_likelihood(optimal[0], y[val_idx], nn_val_pred_conf)
    optimal_conf = laplace_log_likelihood(optimal[0], y[val_idx], lgb_val_predict[:, 1])
    optimal_conf_updated = laplace_log_likelihood(optimal[0], y[val_idx], np.mean([lgb_val_predict[:, 1], nn_val_pred_conf], axis = 0))
    optimal_conf_updated_ngb = laplace_log_likelihood(optimal[0], y[val_idx], np.mean([lgb_val_predict[:, 1], nn_val_pred_conf, ngb_val_conf], axis = 0))
    print("Optimal loss:", optimal_loss, optimal_conf, optimal_conf_updated, optimal_conf_updated_ngb)
    print("Optimal coefficients:", optimal[1])
    
    subm_predict = net.predict(ze2, batch_size=BATCH_SIZE, verbose=0)
    subm_fvc = np.array([subm_predict[:, 1], predicted[:, 0], el_subm, ngb_subm_pred]).T.dot(optimal[1])
    subm_conf = np.mean([subm_predict[:, 2] - subm_predict[:, 0], predicted[:, 1], ngb_subm_conf], axis = 0)
    subm_predict = np.array([subm_fvc, subm_conf]).T
  
    pe1 += subm_predict / NFOLD_MODELS
    
    print()

In [ ]:
sigma_opt = mean_absolute_error(y, pred[:, 1])
unc = pred[:,2] - pred[:, 0]
sigma_mean = np.mean(unc)
print(sigma_opt, sigma_mean)

### PREDICTION

In [ ]:
sub.head()

In [ ]:
sub['FVC1'] = pe1[:, 0]
conf = pe1[:, 1]
sub['Confidence1'] = conf # np.minimum(conf, conf.mean())

In [ ]:
subm = sub[['Patient_Week','FVC','Confidence','FVC1','Confidence1']].copy()

In [ ]:
subm.loc[~subm.FVC1.isnull()].head(10)

In [ ]:
subm.loc[~subm.FVC1.isnull(),'FVC'] = subm.loc[~subm.FVC1.isnull(),'FVC1']
if sigma_mean<70:
    subm['Confidence'] = sigma_opt
else:
    subm.loc[~subm.FVC1.isnull(),'Confidence'] = subm.loc[~subm.FVC1.isnull(),'Confidence1']

In [ ]:
subm.head()

In [ ]:
subm.describe()

In [ ]:
otest = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')
for i in range(len(otest)):
    subm.loc[subm['Patient_Week']==otest.Patient[i]+'_'+str(otest.Weeks[i]), 'FVC'] = otest.FVC[i]
    subm.loc[subm['Patient_Week']==otest.Patient[i]+'_'+str(otest.Weeks[i]), 'Confidence'] = 0.1

In [ ]:
subm[["Patient_Week","FVC","Confidence"]].to_csv("submission.csv", index=False)

In [ ]:
subm